# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.75,82,60,8.57,PN,1703807964
1,1,ta`u,-14.2336,-169.5144,27.83,74,100,1.45,AS,1703807965
2,2,sinan,34.8262,126.1086,5.69,67,26,3.41,KR,1703807966
3,3,kampong cham,11.9934,105.4635,23.55,80,7,4.11,KH,1703807967
4,4,flin flon,54.7682,-101.8650,-5.09,80,0,1.03,CA,1703807969


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_copy = city_data_df.copy()

#Criteria
#Cloudiness = 0 & Wind speed < 4.5 m/s & 21 Degrees < Max Temp < 27 Degrees
city_data_filtered_df = city_data_copy.loc[
    (city_data_copy['Max Temp']<27)
    & (city_data_copy['Max Temp']>21)
    & (city_data_copy['Wind Speed']<4.5)
    & (city_data_copy['Cloudiness']==0)
]

# Drop any rows with null values
city_data_clean_df = city_data_filtered_df.dropna()

# Display sample data
city_data_clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,hawaiian paradise park,19.5933,-154.9731,26.73,74,0,4.12,US,1703807978
163,163,ixtapa,20.7000,-105.2000,25.57,53,0,3.60,MX,1703808161
320,320,saint croix,17.7397,-64.7388,23.31,75,0,2.06,VI,1703808337
328,328,jacutinga,-22.2856,-46.6122,22.16,57,0,1.77,BR,1703808345
425,425,atar,20.5169,-13.0499,22.46,10,0,3.14,MR,1703808454


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_clean_df, columns =
                       ['City',
                       'Country',
                       'Lat',
                       'Lng',
                       'Humidity',
                       ])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,hawaiian paradise park,US,19.5933,-154.9731,74,
163,ixtapa,MX,20.7000,-105.2000,53,
320,saint croix,VI,17.7397,-64.7388,75,
328,jacutinga,BR,-22.2856,-46.6122,57,
425,atar,MR,20.5169,-13.0499,10,
523,bucerias,MX,20.7500,-105.3333,53,
542,san luis de la loma,MX,17.2706,-100.8939,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel

params = {
    'radius' : 10000,
    'categories' : 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
saint croix - nearest hotel: The Buccaneer
jacutinga - nearest hotel: Hotel Parque das Primaveras
atar - nearest hotel: فندق سكليل
bucerias - nearest hotel: Royal Decameron Complex
san luis de la loma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
13,hawaiian paradise park,US,19.5933,-154.9731,74,No hotel found
163,ixtapa,MX,20.7000,-105.2000,53,Ma. Cristina
320,saint croix,VI,17.7397,-64.7388,75,The Buccaneer
328,jacutinga,BR,-22.2856,-46.6122,57,Hotel Parque das Primaveras
425,atar,MR,20.5169,-13.0499,10,فندق سكليل
523,bucerias,MX,20.7500,-105.3333,53,Royal Decameron Complex
542,san luis de la loma,MX,17.2706,-100.8939,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)